#Shocked Iron Sound Speed from 0.5-3 TPa <br>
This project shows the sound speed analysis from this published work: https://doi.org/10.1103/PhysRevB.109.184311

first, import the numpy packages we will need

In [43]:
import numpy as np
import pandas as pd
import re
import ipywidgets as widgets
from IPython.display import display

Next, define a class to hold all of our methods related to the sound speed experiment

In [85]:
class experiment():
    def __init__(self,Us_qz,Us_fe_avg,Us_fe,F_value,P,rho):
        #tell the program the quartz shock velocity (from table I) 
        self.Us_qz=Us_qz
        #tell the program the average iron shock velocity (from table I)
        self.Us_fe_avg=Us_fe_avg
        #tell the program the nonsteady waves corrected iron shock velocity (from table I)
        self.Us_fe=Us_fe
        #tell the program the F value for nonsteady waves (from table I)
        self.F_value=F_value
        #tell the program the average pressure and density for iron (from table III)
        self.P=P
        self.rho=rho
        # Initial Densities in kg/m^3
        self.rho0 = {"qz": 2649, "fe": 7874}
        #These are the parameters for the bilinear fit to iron shock velocity as a function of particle velocity
        #see paper for details
        self.S1=1.46
        self.c0_1=4.54
        self.S2=1.26
        self.c0_2=5.82


    def calc_qz_snd_spd(self,rho_qz):
        #because the tabular equation of state data is export controlled, just fit to the output
        #this is a linear fit to the sound speed as a function of density for singly shocked quartz 
        return (4.393*rho_qz)-12.404

    def calc_qz_reshock(self,upusher):
        #because the tabular equation of state data is export controlled, just fit to the output
        #first, a linear fit to reshocked particle velocity in quartz as a function of the singly shocked
        #quartz shock velocity
        ureshock=0.62*upusher-3.87
        #next, a linear fit to the natural log of the reshocked quartz pressure as a function of the
        #natural log of the pusher shock velocity (natural logarithms compress data to be more linear;
        #great for simplicity, but often detail and nuance is lost)
        preshock=np.exp(2.232*np.log(upusher)+0.2745)
        #a linear fit to the reshocked quartz density as a function of the pusher shock velocity
        rhoreshock=0.1795*upusher+4.3355
        #a linear fit ot the sound speed in quartz as a functino of the density of reshocked quartz
        qz_reshock_sound_speed=(2.824*rhoreshock)-3.447
        return ureshock,preshock,rhoreshock,qz_reshock_sound_speed

    def calc_sample_snd_spd(self):
        #to calculate the sound speed in the iron sample, you need quantities from the other functions in this class
        Usfit=self.Us_Up_fit('fe')[1]
        uparray=self.Us_Up_fit('qz')[0]

        Ppush=self.qz_IM()[2]
        upusher=self.qz_IM()[0]
        rhopush=self.qz_IM()[1]
        Pwitness=self.qz_IM()[5]
        uwitness=self.qz_IM()[3]
        rhowit=self.qz_IM()[4]

        uqzreshock=self.calc_qz_reshock(self.Us_qz)[0]
        reshockpres=self.calc_qz_reshock(self.Us_qz)[1]
        reshockrho=self.calc_qz_reshock(self.Us_qz)[2]
        creshockedqz1=self.calc_qz_reshock(self.Us_qz)[3]

        cwitness1=self.calc_qz_snd_spd(rhowit)
        cpush1=self.calc_qz_snd_spd(rhopush)

        #the Mach numbers and their relationships to the transmission coefficients and sound speed
        #are explained in the main text of the paper
        Mpush=(reshockpres-Ppush)/(rhopush*cpush1*(upusher-uqzreshock))
        Mreshock=(reshockpres-Ppush)/(reshockrho*creshockedqz1*(upusher-uqzreshock))
        Mwitness = (Pwitness) / (rhowit  * cwitness1 * uwitness)
        OpaqueF = 1/self.F_value
        intermediatereshock=(1+Mreshock)/(1+Mpush)
        reshockTC=intermediatereshock
        Mwitness = (Pwitness) / (rhowit  * cwitness1 * uwitness)
        Msample = 1 - (((1 - Mwitness) / OpaqueF) * reshockTC)
        #There's a tricky bit here: when you're calculating the Mach numbers and transmission coefficients
        #you want to take into account the actual reshock state of the quartz pusher, from the nonsteady waves
        #impedance matching calculation. But, when you're loooking at the sample quantities, you want to use 
        #an averaged value, because the sample is opaque and the measure of sound speed extracted is averaged over the 
        #entire time spent in the opaque sample. To that end, the above quantities are calculated for the impedance matched 
        #state for reshocked and singly shocked quartz, and the below quantities use average values for iron
        usample_avg=uparray[np.argwhere(self.Us_fe_avg<=Usfit)[0][0]]
        csample1 = (self.P) / (self.rho * usample_avg * Msample)
        return csample1,usample_avg, self.P, self.rho, Msample, Mwitness, Mreshock, Mpush, OpaqueF,reshockTC

    def calc_gruneisen_parameter(self):
        #this is the equation to calculate Gruneisen parameter from sound speed and a fit to the Hugoniot 
        gammashot = (2/self.rho)*((self.calc_sample_snd_spd()[0] ** 2)*(self.rho**2) - self.dPdRho()*(self.rho**2)) / (( self.P ) -  self.dPdRho() * (self.rho**2) * (-(1 / self.rho) + (1 / (self.rho0['fe'] * 10 ** (-3)))))
        return gammashot
    
    def Us_Up_fit(self,mat):
        uparray=np.linspace(0.00, 49.9, 10000)
        #for quartz witness and pusher, use a linear fit to the LANL sjostrom and crockett data
        #Reference 51 in the paper
        #this linear fit is not super accurate
        if mat=="qz":
            m=1.265
            b=4.828
            Usfit=(m*uparray)+b
            Pfit=self.rho0[mat]*Usfit*uparray
            #wrote this function to be applicable for a bilinear fit, but for the quartz
            #just used a linear fit, so disregard the part2 stuff
            part1=np.nan
            part2=np.nan
            breakpoint=np.nan
        #for iron, use the bilinear fit reported in the main text
        if mat=="fe":
            #identified breakpoint in particle velocity in main text
            breakpoint=np.argwhere(uparray>=6.0)[0][0]
            part1=(uparray[:breakpoint]*self.S1)+self.c0_1
            part2=(uparray[breakpoint:]*self.S2)+self.c0_2
            Usfit=np.concatenate((part1,part2))
            Pfit=self.rho0[mat]*Usfit*uparray
        return uparray,Usfit,Pfit,part1,part2,breakpoint
    
    def qz_IM(self):
        #quartz impedance matching function
        #for singly shocked quartz in the pusher and the witness
        #see paper for details on impedance matching
        uparray=self.Us_Up_fit('qz')[0]
        Usfit=self.Us_Up_fit('qz')[1]
        upusher=uparray[np.argwhere(self.Us_qz<=Usfit)][0][0]
        rhopush = ((self.rho0['qz']*10**(-3)) / (1 - (upusher / self.Us_qz)))
        Ppush=(self.rho0['qz']*10**(-3))*self.Us_qz*upusher
        #make the witness density and particle velocity and pressure the same as the pusher
        #in reality the witness sometimes is a little different from the pusher as it decays or there
        #are modulations in the shock velocity, but for the level of accuracy of this code it doesn't matter
        uwitness=upusher
        rhowitness=rhopush
        Pwitness=Ppush

        return upusher,rhopush,Ppush,uwitness,rhowitness,Pwitness

    
    def dPdRho(self):
        #This is a function to get the bilinear fit to shock velocity as a function of particle velocity for iron
        #translate it into the pressure-density space, then take the derivative of that Hugoniot
        #This derivaive (dPdRho) is used in the calculation of Gruneisen parameter. 
        bilinearvoluparray1=(self.Us_Up_fit('fe')[3] - self.Us_Up_fit('fe')[0][:self.Us_Up_fit('fe')[5]]) / (self.rho0['fe'] * 10 ** (-3) * self.Us_Up_fit('fe')[3])
        bilinearvoluparray2 = (self.Us_Up_fit('fe')[4] - self.Us_Up_fit('fe')[0][self.Us_Up_fit('fe')[5]:]) / (self.rho0['fe'] * 10 ** (-3) * self.Us_Up_fit('fe')[4])
        # bilinear dp/drho
        liquidgbilinear1 = np.polynomial.polynomial.Polynomial([0, -(self.c0_1 ** 2) * ((self.rho0['fe']*10**(-3)) ** 2), (self.c0_1 ** 2) * ((self.rho0['fe']*10**(-3)))])
        liquidhbilinear1 = np.polynomial.polynomial.Polynomial([(self.S1 ** 2) * ((self.rho0['fe']*10**(-3)) ** 2), 2 * self.S1 * (self.rho0['fe']*10**(-3)) - 2 * (self.S1 ** 2) * (self.rho0['fe']*10**(-3)),1 - 2 * self.S1 + (self.S1) ** 2])
        # do the quotient rule to take the derivative of this terrible expression (ratio of polynomials)
        liquidgprimebilinear1 = liquidgbilinear1.deriv(1)
        liquidhprimebilinear1 = liquidhbilinear1.deriv(1)
        liquidhsquaredbilinear1 = liquidhbilinear1 ** 2
        bilinear1dpdrhonum = liquidgprimebilinear1 * liquidhbilinear1 - liquidgbilinear1 * liquidhprimebilinear1
        bilinear1dpdrhodenom = liquidhsquaredbilinear1
        bilinear1dpdrho = bilinear1dpdrhonum(1 / bilinearvoluparray1) / bilinear1dpdrhodenom(1 / bilinearvoluparray1)

        # bilinear dp/drho
        liquidgbilinear2 = np.polynomial.polynomial.Polynomial([0, -(self.c0_2 ** 2) * ((self.rho0['fe']*10**(-3)) ** 2), (self.c0_2 ** 2) * (self.rho0['fe']*10**(-3))])
        liquidhbilinear2 = np.polynomial.polynomial.Polynomial([(self.S2 ** 2) * ((self.rho0['fe']*10**(-3)) ** 2), 2 * self.S2  * (self.rho0['fe']*10**(-3)) - 2 * (self.S2 ** 2) * (self.rho0['fe']*10**(-3)),1 - 2 * self.S2 + (self.S2) ** 2])
        # do the quotient rule to take the derivative of this terrible expression (ratio of polynomials)
        liquidgprimebilinear2 = liquidgbilinear2.deriv(1)
        liquidhprimebilinear2 = liquidhbilinear2.deriv(1)
        liquidhsquaredbilinear2 = liquidhbilinear2 ** 2
        bilinear2dpdrhonum = liquidgprimebilinear2 * liquidhbilinear2 - liquidgbilinear2 * liquidhprimebilinear2
        bilinear2dpdrhodenom = liquidhsquaredbilinear2
        bilinear2dpdrho = bilinear2dpdrhonum(1 / bilinearvoluparray2) / bilinear2dpdrhodenom(1 / bilinearvoluparray2)
        #identified breakpoint in particle velocity in main text
        if self.calc_sample_snd_spd()[1] <= 6.0:
            argslope = int(np.argwhere(1 / bilinearvoluparray1 >= self.rho)[0][0])
            slopeshot = bilinear1dpdrho[int(argslope)]
        if self.calc_sample_snd_spd()[1]>6.0:
            argslope = int(np.argwhere(1 / bilinearvoluparray2 > self.rho)[0][0])
            slopeshot = bilinear2dpdrho[int(argslope)]
        
        return slopeshot


Finally, create an instance of the class with the methods we created, and choose input values from the tables in the paper. 

In [ ]:
#use the ipywidgets library to take user input and output from a jupyter notebook 
output = widgets.Output()
text_input = widgets.Text(
    value='',  # Initially empty
    description='Please choose a shot number from the publication:',  
    disabled=False  #to be editable
)

# Function to handle changes in the text input
def on_text_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            output.clear_output() # from 'Output widgets: leveraging Jupyter’s display system' https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
            #these are the tables from the paper in excel format
            table1 = pd.read_excel('Table1_IronSndSpd.xlsx')
            table2 = pd.read_excel('Table2_IronSndSpd.xlsx')
            table3 = pd.read_excel('Table3_IronSndSpd.xlsx')
            #tell pandas to sort the tables by shot number instead of column heading
            table1=table1.set_index("Shot No.")
            table2=table2.set_index("Shot No.")
            table3=table3.set_index("Shot No.")
            #change the values in the tables from strings to floats so we can interpret them as numbers
            F_scaling=table1.loc[float(change['new'])]["F"]
            Us_qz=table1.loc[float(change['new'])]["Us,Qz (km/s)"]
            Us_fe=table2.loc[float(change['new'])]["Us,Fe (km/s)"]
            Us_fe_avg=table1.loc[float(change['new'])]["< Us,Fe > (km/s)"]
            P_fe_avg=table3.loc[float(change['new'])]["<P> (GPa)"]
            rho_fe_avg=table3.loc[float(change['new'])]["<rho> (g/cc)"]
            #find and separate the values both inside the parenthesis and outside the parenthesis
            #first value is nominal value, second is uncertainty 
            F_scalingvalue=float(re.findall(r'\(.*?\)|[^\(\)]+',F_scaling)[0])
            Us_qzvalue=float(re.findall(r'\(.*?\)|[^\(\)]+',Us_qz)[0])
            Us_fevalue=float(re.findall(r'\(.*?\)|[^\(\)]+',Us_fe)[0])
            Us_fe_avgvalue=float(re.findall(r'\(.*?\)|[^\(\)]+',Us_fe_avg)[0])
            P_fe_avgvalue=float(re.findall(r'\(.*?\)|[^\(\)]+', P_fe_avg)[0])
            rho_fe_value=float(re.findall(r'\(.*?\)|[^\(\)]+', rho_fe_avg)[0])   
            #call the class "experiment" with the initialzation values from the tables
            user_output=experiment(Us_qzvalue,Us_fe_avgvalue,Us_fevalue,F_scalingvalue,P_fe_avgvalue,rho_fe_value)
            #call the sound speed function that belongs to the experiment class we just called
            sound_speed=user_output.calc_sample_snd_spd()
            gruneisen=user_output.calc_gruneisen_parameter()
            #display the value for sound speed (which is the first output of that function)
            print(f"Chosen shot number: {change['new']}")
            print(f"Average pressure: {P_fe_avgvalue}")
            print(f"Average density: {rho_fe_value}")
            print(f"F: {F_scalingvalue}")
            print(f"Average sound speed: {sound_speed[0]}")
            print(f"Gruneisen parameter: {gruneisen}")

          

# Observe changes in the text input widget
text_input.observe(on_text_change, names='value')


display(text_input,output)


Text(value='', description='Please choose a shot number from the publication:')

Output()

You can see that in some instances, this code reproduces the sound speed and Gruneisen parameter fairly well, however, because of the linear fits used for the quartz reshock model, some of the results differ from those listed in the paper by more than a standard deviation. This points to the complex nature of the hydrodynamics involved in this type of experiment and illustrates the need to do a more in depth calculation of the reshocked quartz parameters (which is done in the paper but not reproduced here for export control reasons.) This basic code shows the steps to calculate Mach numbers and transmission coefficients as described in the paper. 